In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
import keras
import random
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import inception_v3 as inception
import vgg16 as VGG
import prepare.collect as pc
'''
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))
'''
N_CLASSES = 2
IMSIZE = (224, 224)

XML_DIR = "../data/annotations/xmls/"
IMG_DIR = "../data/images/"
VAL_RATIO = 0.3

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
# train_dir = '../data/catdog/train'
# Testing directory
# test_dir = '../data/catdog/validation'


# Start with an Inception V3 model, not including the final softmax layer.
base_model = VGG.VGG16(weights='imagenet')
print ('Loaded vgg16 model')

Using TensorFlow backend.


K.image_dim_ordering: tf
Loaded vgg16 model


In [2]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
    

# Add on new fully connected layers for the output classes.
# x = Dense(1024, activation='relu')(base_model.get_layer('fc2').output)
# x = Dropout(0.5)(x)
# predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)
base_model_last = base_model.get_layer('flatten').output
x = Dense(4096, activation='relu', name='fc1-1')(base_model_last)
x = Dense(4096, activation='relu', name='fc1-2')(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)

# y = Dense(4096, activation='relu', name='fc2-1')(base_model_last)
# y = Dense(4096, activation='relu', name='fc2-2')(y)
# aux_predictions = Dense(4, name='aux_predictions')(y)

model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
# Show some debug output
print (model.summary())

print ('Trainable weights')
#model.save_weights('catdog_pretrain.h5')
#print (model.trainable_weights)

In [3]:
xmlFiles = pc.listAllFiles(XML_DIR)
infoList = list(map(lambda f:pc.getInfoTupleForXml(f,IMG_DIR) ,xmlFiles))

random.shuffle(infoList)
cutIndex = int(len(infoList)*VAL_RATIO)
train_files=infoList[:cutIndex]
val_files = infoList[cutIndex:]


In [4]:

#print(val_files)
np.random.seed()
img_datagen = ImageDataGenerator(rescale=1./255)

def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

def my_load_img(img_path,img_datagen,size):
    img = image.load_img(img_path, target_size=size)
    x = image.img_to_array(img)

#     x = img_datagen.img_to_array(img)
    x = img_datagen.random_transform(x)
    x = img_datagen.standardize(x)
    #x = np.expand_dims(x, axis=0)
    return x
def my_img_generator(files,img_datagen,batch_size):
#     index_array = np.random.permutation(len(files))
    
    index = 0
    count = 0
    img_datas=[]
    img_labels=[]
    while 1:
        # create numpy arrays of input data
        # and labels, from each line in the file
        if count < batch_size:
                img_datas.append(my_load_img(files[index][1],img_datagen,IMSIZE))
#                 lable=[0.0,0.0]
#                 lable[files[index][1]]=1.0
                img_labels.append(files[index][2])
                index=(index+1)%len(files)
                count+=1
        else:
            count=0
            #print(img_datas)
            one_hot_labels=to_categorical(img_labels, num_classes=2)
            yield (np.array(img_datas),np.array(one_hot_labels))
            img_datas = []
            img_labels = []
#             random.shuffle(files)
            

batch_size=32
# t = next(my_img_generator(train_files,img_datagen,batch_size))

# model.load_weights('catdog_pretrain_nf.h5') 
# train_data
# train_data.shape
my_train_generator = my_img_generator(train_files,img_datagen,batch_size)
my_val_generator = my_img_generator(val_files,img_datagen,batch_size)

#train_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#         train_dir,  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')

#test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#         test_dir,  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')


# print(a[1].shape)
# print(a[1])

In [6]:

# my_train_generator = my_img_generator(train_files,img_datagen,32)
# my_val_generator = my_img_generator(val_files,img_datagen,32)
# model.fit_generator(
#         my_train_generator,
#         samples_per_epoch=128,
#         nb_epoch=10,
#         validation_data=test_datagen,
#         verbose=2,
#         nb_val_samples=128)
model.fit_generator(
        my_train_generator,
        samples_per_epoch=128,
        nb_epoch=10,
        validation_data=my_val_generator,
        verbose=2,
        nb_val_samples=128)

Epoch 1/10
4s - loss: 0.4679 - acc: 0.8047 - val_loss: 0.5358 - val_acc: 0.7578
Epoch 2/10
3s - loss: 0.5592 - acc: 0.6875 - val_loss: 0.5218 - val_acc: 0.8438
Epoch 3/10
3s - loss: 0.5300 - acc: 0.7422 - val_loss: 0.4943 - val_acc: 0.8281
Epoch 4/10
3s - loss: 0.4265 - acc: 0.8203 - val_loss: 0.3984 - val_acc: 0.8125
Epoch 5/10
3s - loss: 0.3731 - acc: 0.8359 - val_loss: 0.5175 - val_acc: 0.7344
Epoch 6/10
3s - loss: 0.4237 - acc: 0.7734 - val_loss: 1.3898 - val_acc: 0.6875
Epoch 7/10
3s - loss: 0.7147 - acc: 0.7812 - val_loss: 0.3931 - val_acc: 0.8281
Epoch 8/10
3s - loss: 0.3384 - acc: 0.8438 - val_loss: 0.4568 - val_acc: 0.8281
Epoch 9/10
3s - loss: 0.5387 - acc: 0.7109 - val_loss: 0.5457 - val_acc: 0.6484
Epoch 10/10
3s - loss: 0.3731 - acc: 0.8594 - val_loss: 0.3848 - val_acc: 0.8516


In [ ]:
#model.load_weights('catdog_pretrain_nf.h5') 
# Data generators for feeding training/testing images to the model.
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=128,
        nb_epoch=10,
        validation_data=test_generator,
        verbose=2,
        nb_val_samples=128)
#model.save_weights('catdog_pretrain_nf.h5')  # always save your weights after training or during training




img_path = '../data/sport3/validation/hockey/img_2997.jpg'
#img_path = '../data/catdog/test/2.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)

In [ ]:
#model.load_weights('catdog_pretrain.h5') 
#img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img_path = '../data/cat2.jpg'
img_path = '../data/catdog/test/58.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)